In [5]:
!pip install openai vllm==0.8.2

  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)
  Using cached torch-2.6.0-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.21.5-py3-none-manylinux2014_x86_64.whl

In [1]:
import pandas as pd
import json
import re
from typing import List
from openai import OpenAI
from tqdm import tqdm
from transformers import AutoTokenizer
from datasets import load_from_disk
from vllm import LLM, SamplingParams

INFO 06-15 16:11:57 [__init__.py:239] Automatically detected platform cuda.


In [2]:
vllm_model = LLM(
    model="iamjoon/qwen2-7b-rag-ko-checkpoint-285",
    dtype="bfloat16",
)

WARNING 06-15 16:11:58 [config.py:2614] Casting torch.float16 to torch.bfloat16.
INFO 06-15 16:12:06 [config.py:585] This model supports multiple tasks: {'classify', 'score', 'embed', 'generate', 'reward'}. Defaulting to 'generate'.
INFO 06-15 16:12:06 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-15 16:12:08 [core.py:54] Initializing a V1 LLM engine (v0.8.2) with config: model='iamjoon/qwen2-7b-rag-ko-checkpoint-285', speculative_config=None, tokenizer='iamjoon/qwen2-7b-rag-ko-checkpoint-285', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasonin

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

INFO 06-15 16:14:28 [weight_utils.py:281] Time spent downloading weights for iamjoon/qwen2-7b-rag-ko-checkpoint-285: 137.856358 seconds


model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 06-15 16:14:41 [loader.py:447] Loading weights took 12.99 seconds
INFO 06-15 16:14:42 [gpu_model_runner.py:1186] Model loading took 14.2487 GB and 152.313211 seconds
INFO 06-15 16:14:48 [backends.py:415] Using cache directory: /root/.cache/vllm/torch_compile_cache/0fe1359ac4/rank_0_0 for vLLM's torch.compile
INFO 06-15 16:14:48 [backends.py:425] Dynamo bytecode transform time: 6.52 s
INFO 06-15 16:14:52 [backends.py:132] Cache the graph of shape None for later use
INFO 06-15 16:15:14 [backends.py:144] Compiling a graph for general shape takes 25.80 s
INFO 06-15 16:15:29 [monitor.py:33] torch.compile takes 32.32 s in total
INFO 06-15 16:15:29 [kv_cache_utils.py:566] GPU KV cache size: 953,616 tokens
INFO 06-15 16:15:29 [kv_cache_utils.py:569] Maximum concurrency for 32,768 tokens per request: 29.10x
INFO 06-15 16:15:52 [gpu_model_runner.py:1534] Graph capturing finished in 23 secs, took 0.48 GiB
INFO 06-15 16:15:52 [core.py:151] init engine (profile, create kv cache, warmup model) 

In [3]:
test_dataset = load_from_disk('test_dataset')
tokenizer = AutoTokenizer.from_pretrained("iamjoon/qwen2-7b-rag-ko-checkpoint-285")

In [5]:
prompt_lst = []
label_lst = []
questions = []
contexts = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1].split('<|im_end|>')[0]
    question = text.split('<|im_start|>user')[1].split('<|im_end|>')[0].strip()
    context = text.split('검색 결과:\n-----')[1].split('<|im_end|>')[0].strip()
    prompt_lst.append(input)
    label_lst.append(label)
    questions.append(question)
    contexts.append(context)

In [7]:
questions[10]

'리자베타가 꿈 속에서 들었던 장소에 실제로 찾아가 발견한 것은?'

In [8]:
label_lst[10]

'\n리자베타는 꿈속에서 남자가 나타나 억울한 사연을 말하자, 꿈속에서 말한 장소에 가서 땅을 파보니 남자의 시체를 발견했습니다. 리자베타는 시체를 끌고 갈 힘이 없어서 머리통만 잘라서 들고 갔습니다. 이후 그녀는 남자의 머리를 통에 집어 넣고 흙으로 묻어 주었고, 그 자리에서 아름다운 꽃이 피어났습니다 [[ref2]].'

In [9]:
# 생성 파라미터 설정
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=1024,
)

In [10]:
preds = vllm_model.generate(prompt_lst[:50], sampling_params)

Processed prompts: 100%|██████████| 50/50 [00:15<00:00,  3.32it/s, est. speed input: 7252.43 toks/s, output: 562.13 toks/s]


In [11]:
preds = [pred.outputs[0].text for pred in preds]

In [12]:
# OpenAI API 초기화
client = OpenAI(api_key="여러분의 키 값")  # 본인의 API 키로 교체하세요

## LLM 기반의 평가

In [13]:
def evaluate_rag_with_llm(questions, contexts, predictions, labels):
    """
    LLM 평가 메트릭으로 RAG 시스템 평가
    
    Args:
        questions: 질문 리스트
        contexts: 검색 결과 리스트
        predictions: 예측 리스트
        labels: 레이블 리스트
    
    Returns:
        평가 결과가 포함된 데이터프레임
    """
    
    # 결과를 저장할 리스트
    results = []
    
    # 평가 프롬프트
    prompt_template = """
당신은 RAG(Retrieval-Augmented Generation) 시스템 평가 전문가입니다. 아래 정보를 바탕으로 생성된 답변의 품질을 철저히 평가해주세요.

질문: {question}

검색된 컨텍스트:
{context}

생성된 답변:
{prediction}

참조 답변(정답):
{label}

다음 4가지 평가 기준으로 1-5점 척도로 점수를 매겨주세요:

1. 응답 정확성 (Answer Correctness) [1-5]:
   * 생성된 답변이 참조 답변과 비교하여 정확하고 완전한 정보를 제공하는지 평가
   * 1점: 완전히 잘못된 정보
   * 2점: 부분적으로 관련된 정보를 담고 있으나 대부분 부정확함
   * 3점: 정확한 정보와 부정확한 정보가 혼재되어 있음
   * 4점: 대부분 정확하지만 일부 정보가 누락되거나 미미한 오류가 있음
   * 5점: 참조 답변과 비교했을 때 완전히 정확하고 포괄적인 정보를 제공함

2. 컨텍스트 관련성 (Context Relevance) [1-5]:
   * 검색된 컨텍스트가 질문에 대답하기 위해 관련성이 높은지 평가
   * 1점: 컨텍스트가 질문과 전혀 관련이 없음
   * 2점: 컨텍스트가 질문과 간접적으로만 관련됨
   * 3점: 컨텍스트 중 일부만 질문과 직접적으로 관련됨
   * 4점: 대부분의 컨텍스트가 질문과 직접적으로 관련됨
   * 5점: 모든 컨텍스트가 질문에 완벽하게 관련되어 있고 불필요한 정보가 없음

3. 컨텍스트 충실성 (Context Faithfulness) [1-5]:
   * 생성된 답변이 주어진 컨텍스트에만 기반하는지, 아니면 없는 정보를 추가했는지 평가
   * 1점: 답변이 컨텍스트에 없는 정보로만 구성됨 (심각한 환각)
   * 2점: 답변이 주로 컨텍스트에 없는 정보로 구성됨
   * 3점: 답변이 컨텍스트 정보와 없는 정보가 혼합되어 있음
   * 4점: 답변이 주로 컨텍스트에 기반하지만 약간의 추가 정보가 있음
   * 5점: 답변이 전적으로 컨텍스트에 있는 정보만을 사용함

4. 컨텍스트 충분성 (Context Recall) [1-5]:
   * 검색된 컨텍스트가 질문에 완전히 답변하기에 충분한 정보를 포함하는지 평가
   * 1점: 컨텍스트가 답변에 필요한 정보를 전혀 포함하지 않음
   * 2점: 컨텍스트가 필요한 정보의 일부만 포함함
   * 3점: 컨텍스트가 필요한 정보의 약 절반을 포함함
   * 4점: 컨텍스트가 필요한 정보의 대부분을 포함하지만 일부 누락됨
   * 5점: 컨텍스트가 질문에 완전히 답변하기 위한 모든 필요한 정보를 포함함

반드시 다음 JSON 형식으로만 응답하세요. 마크다운은 사용하지 않습니다.:
{
  "answer_correctness": 정수로 된 점수(1-5),
  "context_relevance": 정수로 된 점수(1-5),
  "context_faithfulness": 정수로 된 점수(1-5),
  "context_recall": 점수(1-5),
  "analysis": "종합적인 분석 의견"
}

다른 형식의 응답은 하지 마세요. 오직 마크다운이 아닌 JSON만 반환하세요.
"""
    
    # 각 항목에 대해 평가 수행
    for i in tqdm(range(len(questions)), total=len(questions), desc="RAG 평가 진행 중"):
        try:
            # 프롬프트 생성 - format 대신 replace 사용
            prompt = prompt_template
            prompt = prompt.replace("{question}", str(questions[i]) if questions[i] is not None else "")
            prompt = prompt.replace("{context}", str(contexts[i]) if contexts[i] is not None else "")
            prompt = prompt.replace("{prediction}", str(predictions[i]) if predictions[i] is not None else "")
            prompt = prompt.replace("{label}", str(labels[i]) if labels[i] is not None else "")

            # GPT-4 API 호출
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "당신은 RAG 평가 도구입니다. 반드시 유효한 JSON 형식으로만 응답하세요."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0,
                response_format={"type": "json_object"}
            )
            
            # 결과 파싱
            result = json.loads(response.choices[0].message.content)
            
            # 개별 메트릭 점수 추출
            answer_correctness = result['answer_correctness']
            context_relevance = result['context_relevance']
            context_faithfulness = result['context_faithfulness']
            context_recall = result['context_recall']
            
            # 총점 직접 계산 (개별 메트릭의 합)
            total_score = answer_correctness + context_relevance + context_faithfulness + context_recall
            
            # 원본 데이터와 평가 결과 결합
            row_result = {
                'id': i,
                'question': questions[i],
                'answer_correctness': answer_correctness,
                'context_relevance': context_relevance,
                'context_faithfulness': context_faithfulness,
                'context_recall': context_recall,
                'total_score': total_score,
                'analysis': result['analysis']
            }
            
            results.append(row_result)
            
        except Exception as e:
            print(f"항목 {i} 평가 중 오류 발생: {e}")
            results.append({
                'id': i,
                'question': questions[i],
                'error': str(e)
            })
    
    # 결과 데이터프레임 생성
    results_df = pd.DataFrame(results)
    
    # 요약 통계 계산
    if 'total_score' in results_df.columns:
        metrics_summary = {
            '평균 총점': results_df['total_score'].mean(),
            '응답 정확성 평균': results_df['answer_correctness'].mean(),
            '컨텍스트 관련성 평균': results_df['context_relevance'].mean(),
            '컨텍스트 충실성 평균': results_df['context_faithfulness'].mean(),
            '컨텍스트 충분성 평균': results_df['context_recall'].mean()
        }
        print("\n===== 평가 요약 =====")
        for metric, value in metrics_summary.items():
            print(f"{metric}: {value:.2f}")
    
    return results_df, metrics_summary if 'total_score' in results_df.columns else results_df

In [14]:
results_df, metrics_summary = evaluate_rag_with_llm(questions[:50], contexts[:50], preds[:50], label_lst[:50])

RAG 평가 진행 중: 100%|██████████| 50/50 [02:34<00:00,  3.10s/it]


===== 평가 요약 =====
평균 총점: 18.64
응답 정확성 평균: 4.30
컨텍스트 관련성 평균: 4.88
컨텍스트 충실성 평균: 4.52
컨텍스트 충분성 평균: 4.94


In [15]:
results_df.head()

,id,question,answer_correctness,context_relevance,context_faithfulness,context_recall,total_score,analysis
0,0,유아용 전동자동차와 장난감 레고 세트를 경품 추첨으로 주는 아파트는?,1,5,1,5,12,생성된 답변은 질문에 대한 정확한 정보를 제공하지 못했습니다. '김포한강신도시 반도...
1,1,2009년 이혼 소송 접수 건수는 몇 건인가?,5,5,5,5,20,"생성된 답변은 참조 답변과 정확히 일치하며, 검색된 컨텍스트에서 제공된 정보를 정확..."
2,2,규제로 인해 어떤 종류의 매장이 증가했는가?,5,5,5,5,20,생성된 답변은 참조 답변과 비교했을 때 정확하고 포괄적인 정보를 제공하고 있습니다....
3,3,조텍컵에서 입상하면 어떤 플랫폼을 통해 상금을 받을 수 있는가?,5,5,5,5,20,생성된 답변은 참조 답변과 비교했을 때 정확하고 포괄적인 정보를 제공하고 있습니다....
4,4,직무부트캠프를 제공하는 회사를 운영하는 사람의 이름은?,5,5,5,5,20,생성된 답변은 참조 답변과 비교했을 때 정확하고 포괄적인 정보를 제공하고 있습니다....


## 정량 평가


In [16]:
def extract_ref_numbers(text: str) -> List[int]:
    """
    텍스트에서 [[ref숫자]] 패턴의 숫자들을 추출하여 중복 없이 정렬된 리스트로 반환합니다.

    Args:
        text (str): 분석할 텍스트

    Returns:
        List[int]: 추출된 숫자들의 정렬된 리스트

    Example:
        >>> text = "이것은 [[ref1]]과 [[ref2]], [[ref1]]입니다."
        >>> extract_ref_numbers(text)
        [1, 2]
    """
    # [[ref숫자]] 패턴을 찾는 정규표현식
    pattern = r'\[\[ref(\d+)\]\]'

    # 모든 매치를 찾아서 숫자만 추출
    numbers = [int(match.group(1)) for match in re.finditer(pattern, text)]

    # 중복 제거하고 정렬하여 반환
    return sorted(list(set(numbers)))

In [17]:
label = '''제3차 중동 전쟁과 아랍의 겨울의 주요 원인과 결과는 다음과 같습니다.

### 제3차 중동 전쟁 (6일 전쟁)
**주요 원인:**
1. **티란 해협 봉쇄:** 이집트가 이스라엘 선박의 티란 해협 통과를 봉쇄하면서 긴장이 고조되었습니다. 이스라엘은 이를 전쟁의 명분으로 삼았습니다[[ref1]].
2. **군사적 준비:** 이집트는 이스라엘과의 국경에 군을 배치하고, 이스라엘은 이에 대응해 예방적 공습을 감행했습니다[[ref1]].

**주요 결과:**
1. **영토 확장:** 이스라엘은 가자 지구, 시나이 반도, 동예루살렘, 요르단 강 서안 지구, 골란 고원을 점령하여 영토를 3배로 확장했습니다[[ref1]].
2. **난민 발생:** 전쟁으로 인해 약 100,000명의 시리아인과 300,000명의 팔레스타인인이 난민이 되었습니다[[ref1]].
3. **군사적 자만:** 이스라엘의 승리는 군사적 자만을 초래하여 이후 1973년 욤키푸르 전쟁에서 아랍 연합군의 초기 승리를 가능하게 했습니다[[ref1]].

### 아랍의 겨울
**주요 원인:**
1. **아랍의 봄의 실패:** 2011년 아랍의 봄 이후 민주화와 개혁을 목표로 한 시도들이 실패하면서 권위주의가 복고되고, 사회적 불안정이 증가했습니다[[ref4]].
2. **이슬람 극단주의의 부상:** 이슬람 극단주의와 테러 단체들이 활동을 강화하면서 지역 내 폭력과 혼란이 가중되었습니다[[ref4]].

**주요 결과:**
1. **내전과 사회 불안정:** 시리아 내전, 이라크 내전, 예멘 내전 등 여러 지역에서 내전과 사회 불안정이 발생했습니다[[ref4]].
2. **경제적 손실:** 아랍의 겨울로 인해 발생한 사회적 손실비용은 약 8천억 달러에 달하며, 수백만 명의 난민이 발생했습니다[[ref4]].
3. **인도적 위기:** 시리아, 이집트, 이라크 등 여러 지역에서 인도적 지원이 필요한 사람들의 수가 급증했습니다[[ref4]].

이와 같이 제3차 중동 전쟁과 아랍의 겨울은 각각 중동 지역의 군사적, 정치적, 사회적 변화를 초래한 중요한 사건들입니다.'''

In [18]:
cited_docs = extract_ref_numbers(label)
print(cited_docs)

[1, 4]


In [19]:
model_prediction = '''제3차 중동 전쟁과 아랍의 겨울은 각각 다른 시기에 발생한 사건이며, 주요 원인과 결과는 다음과 같습니다.

제3차 중동 전쟁은 1967년 6월에 시작된 이스라엘과 주변 국가인 이집트, 요르단, 시리아, 레바논 사이의 전쟁입니다. 이 전쟁의 주요 원인은 이스라엘과 주변 국가 사이의 긴장과 불안정한 관계였습니다. 이스라엘은 1956년 수에즈 위기 때 주요 목표 중 하나였던 티란 해협을 확보하려고 했고, 이 지역은 1950년 이래 이집트가 이스라엘의 항해를 봉쇄하고 있었습니다. 이스라엘은 이 지역의 봉쇄를 계속해서 유지하려고 했지만, 이는 결국 1967년 6월에 이스라엘과 주변 국가 사이의 전쟁으로 이어졌습니다.

전쟁의 결과로, 이스라엘은 이집트로부터 가자 지구와 시나이 반도를, 요르단으로부터 동예루살렘과 요르단 강 서안 지구를, 시리아로부터 골란 고원을 획득했습니다. 이스라엘의 영토는 3배나 커졌고, 이스라엘군은 군사적 승리를 거두었습니다. 그러나 이 전쟁으로 인해 군사들은 자만감에 빠졌고, 이는 1973년 욤키푸르 전쟁 당시 아랍 연합군이 초기에 승리를 거두는 계기가 되었습니다. 또한, 이 전쟁으로 인해 100,000명의 시리아인이 골란 고원을 떠났고, 300,000명의 팔레스타인인이 서안 지구를 떠났는데 이들은 모두 난민이 되었습니다.

아랍의 겨울은 2011년 아랍의 봄 이후 아랍 일대에서 벌어지고 있는 광범위한 혼란과 폭력과 사회불안정 사태를 가리키는 말입니다. 이 겨울에는 시리아 내전, 이라크 내전, 이집트 위기, 예멘 내전이 모두 포함됩니다. 권위주의가 복고되고 자유 민권이 억압되고 있으며, 이슬람 극단주의와 테러 단체가 곳곳에서 준동하고 있습니다. 아랍의 겨울은 복수의 지역에서 내란과 내전이 발생하고, 사회가 불안정해지며, 아랍 지역의 경제와 인구가 쇠퇴하고 인종주의와 종교적 종파주의가 판을 치는 것으로 특징지어집니다. 아랍의 겨울의 가장 극단적 사례는 시리아입니다. 바샤르 알 아사드 대통령의 독재에 반대하여 일어난 시위에 알누스라 전선 등의 이슬람주의 집단이 개입하고, 자유 시리아군의 부패와 범죄가 밝혀지면서 이슬람주의자들의 입지가 강화되었습니다. 그리하여 시리아는 독재정부와 반정부 반군과 이슬람주의 테러리스트들의 삼파내전에 휩싸이게 되었으며, 그 여파는 이웃나라인 레바논과 이라크까지 번지고 있습니다. [[ref1]]'''

In [20]:
model_pred = extract_ref_numbers(model_prediction)
print(model_pred)

[1]


In [21]:
def calculate_f1_score(labels, predictions):
    """
    F1 score 계산 (multilabel 케이스) - 이중 리스트 입력 지원
    
    Args:
        labels: 샘플별 실제 레이블이 담긴 리스트 또는 이중 리스트
        predictions: 샘플별 예측 레이블이 담긴 리스트 또는 이중 리스트
    
    Returns:
        F1 점수
    """
    # 입력 형식 확인
    is_nested_list = isinstance(labels, list) and len(labels) > 0 and isinstance(labels[0], list)
    
    # 이중 리스트가 아니면 이중 리스트로 변환
    if not is_nested_list:
        labels = [labels]
        predictions = [predictions]
    
    # 길이 확인
    if len(labels) != len(predictions):
        raise ValueError("라벨과 예측의 길이가 일치하지 않습니다.")
    
    # 전체 통계 초기화
    total_true_positives = 0
    total_false_positives = 0
    total_false_negatives = 0
    
    # 각 샘플에 대해 계산
    for sample_labels, sample_preds in zip(labels, predictions):
        # 빈 리스트는 빈 집합으로 변환
        if not sample_labels:
            sample_labels = set()
        else:
            sample_labels = set(sample_labels)
            
        if not sample_preds:
            sample_preds = set()
        else:
            sample_preds = set(sample_preds)
        
        # 통계 계산
        true_positives = len(sample_labels.intersection(sample_preds))
        false_positives = len(sample_preds - sample_labels)
        false_negatives = len(sample_labels - sample_preds)
        
        # 전체 통계에 추가
        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives
    
    # 특수 케이스: 둘 다 빈 리스트인 경우는 F1 = 1.0으로 처리
    if sum(bool(l) for l in labels) == 0 and sum(bool(p) for p in predictions) == 0:
        return 1.0
    
    # 정밀도와 재현율 계산
    precision = total_true_positives / (total_true_positives + total_false_positives) if (total_true_positives + total_false_positives) > 0 else 0
    recall = total_true_positives / (total_true_positives + total_false_negatives) if (total_true_positives + total_false_negatives) > 0 else 0
    
    # F1 점수 계산
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return f1

In [22]:
test_cases = [
    ([1, 4], [1]),  # 기본 케이스
    ([1, 2, 3], [1, 2, 3]),  # 완벽한 예측
    ([1, 2, 3], [4, 5, 6]),  # 완전히 틀린 예측
    ([1], [1, 2, 3]),  # 과대 예측
    ([], []),  # 빈 리스트
]

for labels, predictions in test_cases:
    f1 = calculate_f1_score([labels], [predictions])
    print(f"Labels: {labels}")
    print(f"Predictions: {predictions}")
    print(f"F1 Score: {f1:.4f}")
    print("-" * 30)

Labels: [1, 4]
Predictions: [1]
F1 Score: 0.6667
------------------------------
Labels: [1, 2, 3]
Predictions: [1, 2, 3]
F1 Score: 1.0000
------------------------------
Labels: [1, 2, 3]
Predictions: [4, 5, 6]
F1 Score: 0.0000
------------------------------
Labels: [1]
Predictions: [1, 2, 3]
F1 Score: 0.5000
------------------------------
Labels: []
Predictions: []
F1 Score: 1.0000
------------------------------


In [23]:
label_ref_numbers = []
pred_ref_numbers = []

for label, pred in zip(label_lst[:50], preds[:50]):
    label_ref_numbers.append(extract_ref_numbers(label))
    pred_ref_numbers.append(extract_ref_numbers(pred))

In [24]:
label_ref_numbers[20:30]

[[1], [1], [2], [1], [3], [1, 3], [1], [2], [1], [1]]

In [25]:
pred_ref_numbers[20:30]

[[2], [1], [2], [1], [3], [1, 2, 3], [1], [2], [1], [1]]

In [26]:
f1 = calculate_f1_score(label_ref_numbers, pred_ref_numbers)

In [27]:
print(f1)

0.9473684210526315
